In [46]:
# Importing modules
import pandas as pd
import os
os.chdir('..')
# Read data into papers
papers = pd.read_csv('/content/sample.csv')
# Print head
papers.head()

,Alternate URL,Article Abstract,Author Biography,Banner,Biographical Categories,Body,Byline,Column Name,Column Number,Correction Date,Correction Text,Credit,Dateline,Day Of Week,Descriptors,Feature Page,General Online Descriptors,Guid,Headline,Kicker,Lead Paragraph,Locations,Names,News Desk,Normalized Byline,Online Descriptors,Online Headline,Online Lead Paragraph,Online Locations,Online Organizations,Online People,Online Section,Online Titles,Organizations,Page,People,Publication Date,Publication Day Of Month,Publication Month,Publication Year,Section,Series Name,Slug,Taxonomic Classifiers,Titles,Types Of Material,Url,Word Count,Filename
0,NaN,NaN,NaN,NaN,NaN,"LEVENE-Ricky. So dearly loved, so deeply misse...",NaN,NaN,3,NaN,NaN,The New York Times,NaN,Thursday,NaN,NaN,NaN,1214647,"Paid Notice: Memorials LEVENE, RICKY",NaN,"LEVENE-Ricky. So dearly loved, so deeply misse...",NaN,NaN,Classified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paid Death Notices,NaN,NaN,27,RICKY LEVENE,20000713T000000,13,7,2000,A,NaN,DN071300,Top/Classifieds/Paid Death Notices,NaN,Paid Memorial Notice,http://query.nytimes.com/gst/fullpage.html?res...,18,2000/07/13/1214647.xml
1,NaN,"Volunteer House, light-hearted architectural f...",NaN,NaN,NaN,''We wanted a building with a certain exuberan...,LINDA LEE,NaN,4,NaN,NaN,The New York Times,NaN,Thursday,Parks and Other Recreation Areas,NaN,Leisure|Parks and Other Recreation Areas,1214444,A Bucolic Meeting Place For City Park Volunteers,CURRENTS: ARCHITECTURE,''We wanted a building with a certain exuberan...,New York City|Riverside Park (NYC),NaN,House & Home/Style Desk,"Lee, Linda",NaN,NaN,NaN,NaN,NaN,NaN,Home and Garden; Style,NaN,NaN,3,Linda Lee|Jeffrey Murphy,20000713T000000,13,7,2000,F,NaN,13PARK,Top/Features/Home and Garden|Top/Features/Styl...,NaN,NaN,http://query.nytimes.com/gst/fullpage.html?res...,228,2000/07/13/1214444.xml
2,NaN,Small-business owners and farmers whose Washin...,NaN,NaN,NaN,Small-business owners and farmers whose Washin...,DAVID CAY JOHNSTON,NaN,2,NaN,NaN,The New York Times,NaN,Thursday,Taxation|Wills and Estates|Law and Legislation...,NaN,Election Issues|Wills and Estates|Inheritance ...,1214527,"Despite Benefits, Democrats' Estate Tax Plan G...",NaN,Small-business owners and farmers whose Washin...,United States,NaN,Business/Financial Desk,"Johnston, David Cay",NaN,NaN,NaN,NaN,NaN,NaN,Business,NaN,NaN,1,Bill Clinton|David Cay Johnston|Sanford J Schl...,20000713T000000,13,7,2000,C,NaN,Y13EST$01,Top/News/Business|Top/News/World/Countries and...,NaN,NaN,http://query.nytimes.com/gst/fullpage.html?res...,1400,2000/07/13/1214527.xml
3,NaN,TriBeCa Perfroming Arts Center will host Festi...,NaN,NaN,NaN,"For Indian playwrights, involvement in New Yor...",FELICIA R. LEE,NaN,4,NaN,NaN,The New York Times,NaN,Thursday,Theater|TriBeCa Performing Arts Center (NYC)|F...,NaN,Theater|Festivals,1214463,Festival Opens Door to Indian Theater,NaN,"For Indian playwrights, involvement in New Yor...",NaN,NaN,The Arts/Cultural Desk,"Lee, Felicia R",NaN,NaN,NaN,NaN,NaN,NaN,Arts; Theater,NaN,Indo-American Arts Council,8,Felicia R Lee|Mahesh Dattani|Vijay Tendulkar,20000713T000000,13,7,2000,E,NaN,13FEST$01,Top/News|Top/Features/Arts|Top/Features/Theate...,NaN,NaN,http://query.nytimes.com/gst/fullpage.html?res...,402,2000/07/13/1214463.xml
4,NaN,Warming relations between South Korea and Japa...,NaN,NaN,NaN,As he waited to board a flight back to his nat...,HOWARD W. FRENCH,NaN,1,NaN,NaN,The New York Times,"FUKUOKA, Japan",Thursday,International Relations|Japan-International Re...,NaN,International Relations|Travel and Vacations|P...,1214519,Travel Boom Pulls Japan and South Korea Closer,NaN,As he waited to board a flight back to his nat...,South Korea|Japan|Fukuoka (Japan),NaN,Foreign Desk,"French, Howard W",NaN,NaN,NaN,NaN,NaN,NaN,World,NaN,NaN,3,Howard W French,20000713T000000,13,7,2000,A,NaN,Y13JAP$03,Top/News|Top/News/World/Countries and Territor...,NaN,NaN,http://query.nytimes.com/gst/fullpage.html?res...,1517,2000/07/13/

In [47]:
# Remove the columns
papers = papers.drop(columns=['Alternate URL', 'Article Abstract', 'Author Biography','Banner','Biographical Categories','Byline','Column Name','Column Number','Correction Date','Correction Text','Credit','Dateline','Day Of Week','Descriptors','Feature Page','General Online Descriptors','Guid','Headline','Kicker','Lead Paragraph','Locations','Names','News Desk','Normalized Byline','Online Descriptors','Online Headline','Online Lead Paragraph','Online Locations','Online Organizations','Online People','Online Section','Online Titles','Organizations','Page','People','Publication Date','Publication Day Of Month','Publication Month','Publication Year','Section','Series Name','Slug','Taxonomic Classifiers','Titles','Types Of Material','Url','Word Count','Filename'], axis=1)
# Print out the first rows of papers
papers.head()

,Body
0,"LEVENE-Ricky. So dearly loved, so deeply misse..."
1,''We wanted a building with a certain exuberan...
2,Small-business owners and farmers whose Washin...
3,"For Indian playwrights, involvement in New Yor..."
4,As he waited to board a flight back to his nat...


In [48]:
# Load the regular expression library
import re
# Remove punctuation
papers['paper_text_processed'] = papers['Body'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
papers['paper_text_processed'] = papers['paper_text_processed'].map(lambda x: x.lower())
# Print out the first rows of papers
papers['paper_text_processed'].head()


<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<ipython-input-48-0125e3c9d3ee>:4: DeprecationWarning: invalid escape sequence \.
  papers['paper_text_processed'] = papers['Body'

0    levene-ricky so dearly loved so deeply missed ...
1    ''we wanted a building with a certain exuberan...
2    small-business owners and farmers whose washin...
3    for indian playwrights involvement in new york...
4    as he waited to board a flight back to his nat...
Name: paper_text_processed, dtype: object

In [49]:
%%time
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data = papers.paper_text_processed.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1])

[['levene', 'ricky', 'so', 'dearly', 'loved', 'so', 'deeply', 'missed', 'so', 'much', 'too', 'soon', 'our', 'constant', 'love', 'mom', 'and', 'your', 'family']]
CPU times: user 58.1 ms, sys: 800 µs, total: 58.9 ms
Wall time: 59.1 ms


In [50]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [51]:
# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [52]:

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

[['dearly', 'love', 'deeply', 'miss', 'much', 'soon', 'constant', 'love', 'mom', 'family']]


In [53]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1)]]


In [54]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=6, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [55]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


[(0,
  '0.022*"say" + 0.012*"would" + 0.011*"tax" + 0.009*"farmer" + 0.009*"marble" '
  '+ 0.008*"plan" + 0.008*"estate_tax" + 0.008*"wall" + 0.007*"estate" + '
  '0.007*"light"'),
 (1,
  '0.013*"woman" + 0.008*"say" + 0.008*"japanese" + 0.006*"year" + '
  '0.006*"much" + 0.006*"korean" + 0.006*"many" + 0.005*"country" + '
  '0.004*"would" + 0.004*"book"'),
 (2,
  '0.024*"price" + 0.020*"say" + 0.015*"gas" + 0.010*"oil" + 0.010*"year" + '
  '0.007*"station" + 0.006*"profit" + 0.006*"inventory" + 0.006*"rise" + '
  '0.006*"market"'),
 (3,
  '0.030*"say" + 0.007*"make" + 0.007*"low" + 0.007*"last" + 0.006*"year" + '
  '0.006*"gooden" + 0.005*"team" + 0.005*"start" + 0.005*"people" + '
  '0.005*"time"'),
 (4,
  '0.024*"say" + 0.020*"state" + 0.013*"year" + 0.011*"district" + '
  '0.008*"would" + 0.007*"school" + 0.006*"account" + 0.005*"armory" + '
  '0.005*"official" + 0.004*"proposal"'),
 (5,
  '0.010*"name" + 0.009*"site" + 0.008*"year" + 0.008*"race" + 0.007*"sign" + '
  '0.006*"web" 

In [56]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3166600191192936


In [57]:
!pip install pyLDAvis

import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.007245 -0.042422       1        1  23.305318
4      0.086009 -0.093733       2        1  20.968152
1     -0.009189  0.087596       3        1  18.735645
5     -0.160324 -0.043167       4        1  13.805529
2      0.028538  0.120797       5        1  13.176609
0      0.062211 -0.029071       6        1  10.008748, topic_info=          Term        Freq       Total Category  logprob  loglift
1365     price   32.000000   32.000000  Default  30.0000  30.0000
64         say  174.000000  174.000000  Default  29.0000  29.0000
298      state   49.000000   49.000000  Default  28.0000  28.0000
2892       gas   19.000000   19.000000  Default  27.0000  27.0000
1050  district   22.000000   22.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
77        year    5.346371   75.685139   Topic6  -5.1680  -0.3485
229      owner    3.364224   14.421732   Topic6  -5.6312   0.8462
208       make    3.637005   42.802158   Topic6  -5.5533  -0.1637
53      people    3.377126   27.656842   Topic6  -5.6274   0.1989
306       take    2.726490   25.578928   Topic6  -5.8414   0.0630

[377 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
80        3  0.174821     accord
80        5  0.699283     accord
80        6  0.174821     accord
891       2  0.829902    account
891       4  0.150891    account
...     ...       ...        ...
337       4  0.059494  yesterday
337       5  0.237976  yesterday
337       6  0.059494  yesterday
1891      1  0.725268      young
1891      4  0.181317      young

[719 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 2, 6, 3, 1])